In [2]:
%pip install ipykernel requests pandas matplotlib jinja2 gender-guesser pymupdf spacy


Note: you may need to restart the kernel to use updated packages.


In [3]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import os
import fitz  # PyMuPDF
import re
import spacy
import pandas as pd
from gender_guesser.detector import Detector

# Load spaCy and gender detector
nlp = spacy.load("en_core_web_sm")
gender_detector = Detector()


In [5]:
def extract_script_text(pdf_path):
    with fitz.open(pdf_path) as doc:
        return "\n".join([page.get_text() for page in doc])

def load_scripts_from_folder(folder_path):
    scripts = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            title = filename.replace('.pdf', '')
            text = extract_script_text(os.path.join(folder_path, filename))
            scripts[title] = text
    return scripts

# Update path to your local folder
scripts = load_scripts_from_folder("D:\PROG\docu3C\BollyAI_2_0\Data\scripts-data")
print(f"✅ Loaded {len(scripts)} scripts.")


<>:15: SyntaxWarning: invalid escape sequence '\P'
<>:15: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Aldrin Joan\AppData\Local\Temp\ipykernel_47716\1083483818.py:15: SyntaxWarning: invalid escape sequence '\P'
  scripts = load_scripts_from_folder("D:\PROG\docu3C\BollyAI_2_0\Data\scripts-data")


✅ Loaded 13 scripts.


In [6]:
def extract_intro_lines(text):
    lines = text.split('\n')
    clean_lines = []

    for line in lines:
        line = line.strip()
        if 5 < len(line.split()) < 30:
            if not line.isupper() and not re.match(r'^[A-Z]+:$', line):
                clean_lines.append(line)
    return clean_lines


In [7]:
def detect_character_and_gender(line):
    doc = nlp(line)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            name = ent.text
            gender = gender_detector.get_gender(name.split()[0])
            return name, gender
    return None, None


In [9]:
clean_data = []

for title, text in scripts.items():
    lines = extract_intro_lines(text)
    for line in lines:
        character, gender = detect_character_and_gender(line)
        if character and gender in ['male', 'female']:
            clean_data.append({
                "script": title,
                "character": character,
                "gender": gender,
                "line": line.strip()
            })

df_cleaned = pd.DataFrame(clean_data)
df_cleaned.to_csv("D:\PROG\docu3C\BollyAI_2_0\output\cleaned_script_data_plots.csv", index=False)
print("✅ Saved cleaned data to cleaned_script_data_plots.csv")
df_cleaned.head()


<>:16: SyntaxWarning: invalid escape sequence '\P'
<>:16: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Aldrin Joan\AppData\Local\Temp\ipykernel_47716\322005054.py:16: SyntaxWarning: invalid escape sequence '\P'
  df_cleaned.to_csv("D:\PROG\docu3C\BollyAI_2_0\output\cleaned_script_data_plots.csv", index=False)


✅ Saved cleaned data to cleaned_script_data_plots.csv


,script,character,gender,line
0,Haider,Shiraz,male,detained at the Shiraz cinema camp…
1,Haider,Ghazala,female,Ghazala sees the soldier raising the barricade...
2,Haider,Ghazala,female,Surgical instruments are boiling in a pot. Gha...
3,Haider,Ghazala,female,"silent, Ghazala has a sense of paranoia in her..."
4,Haider,Ghazala,female,in with a bunch of other surgical tools in a t...
